Script Purpose: Scrape resturant reviews from Google Map

Coding Reference: https://www.linkedin.com/pulse/webscrape-google-map-reviews-using-selenium-python-choy-siew-wearn/?trk=pulse-article_more-articles_related-content-card

In [1]:
import pandas as pd
import time

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [2]:
# Initialize WebDriver
driver = webdriver.Chrome()
#driver = webdriver.Chrome(ChromeDriverManager().install())

# Extract Reviews

In [3]:
# Specify the name here
resturant_name = 'pai'

# Replace the URL here
url = 'https://www.google.com/maps/place/Yin+Ji+Chang+Fen+%E9%8A%80%E8%A8%98%E8%85%B8%E7%B2%89+(Downtown)/@43.6556323,-79.4007838,18z/data=!4m15!1m7!3m6!1s0x882b352f899ec8dd:0xc49c4326e9ddc453!2zVGVubWlsZXMgQi5CLlEg5pil6aOO5Y2B6YeMIChEb3dudG93buW6l--8iQ!8m2!3d43.6555915!4d-79.3988178!16s%2Fg%2F11h_x8qp0m!3m6!1s0x882b35fc9d8bba53:0xdb2bbff8043dc64e!8m2!3d43.6558099!4d-79.3988996!9m1!1b1'

driver.get(url)
#time.sleep(5)

In [4]:
# Get the total number of reviews
num_reviews = driver.find_element("xpath","//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]").text
num_reviews = int(num_reviews.split(' ')[0].replace(",",''))
num_reviews

266

In [5]:
# Click Sort button and select "Newest"
# Page Source - Inspect - Locate the button and copy XPATH

driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button').click()
time.sleep(1)

driver.find_element("xpath","//*[@id='action-menu']/div[2]").click()
time.sleep(5)


In [6]:
SCROLL_PAUSE_TIME = 5

number = 0

while True:
    number = number+1

    # Scroll down to bottom    
    ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
    driver.execute_script('arguments[0].scrollBy(0, 5000);', ele)

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    #ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
    #new_height = driver.execute_script("return arguments[0].scrollHeight", ele)

    if number >= num_reviews/10:
        break


In [7]:
item = driver.find_elements("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]')

time.sleep(3)

In [9]:
from selenium.webdriver.common.by import By

name_list = []
time_list = []
stars_list = []
reviews_list = []

for i in item:

    button = i.find_elements(By.TAG_NAME,'button')
    for m in button:
        if m.text == "More":
            m.click()
    time.sleep(5)

    name = i.find_elements(By.CLASS_NAME,"d4r55")
    time = i.find_elements(By.CLASS_NAME,"kvMYJc")
    stars = i.find_elements(By.CLASS_NAME,"wiI7pd")
    reviews = i.find_elements(By.CLASS_NAME,"rsqaWe")

    for j,k,l,p in zip(name,time,stars,reviews):
        name_list.append(j.text)
        time_list.append(p.text)
        stars_list.append(k.get_attribute("aria-label"))
        reviews_list.append(l.text)

In [10]:
review = pd.DataFrame(
    {'name': name_list,
     'time': time_list,
     'stars': stars_list,
     'review': reviews_list,})

review.to_csv('google_review_{}.csv'.format(resturant_name),index=False)
print(review)

                 name         time      stars  \
0        Tashi Jurmey   2 days ago   4 stars    
1          Isaiah Tan   6 days ago   4 stars    
2           Sue Kwong   6 days ago   5 stars    
3      Irene Kittycat   a week ago   4 stars    
4              Krn JE   a week ago   4 stars    
..                ...          ...        ...   
261           Emily K  2 years ago   5 stars    
262          Bill Liu  2 years ago   5 stars    
263  Josephine Nguyen  2 years ago   4 stars    
264       harley yang  2 years ago   4 stars    
265           Mandy K  2 years ago   5 stars    

                                                review  
0                                                       
1    Cheap and tasty eats. Classic congee breakfast...  
2                                                       
3    Stumbled across this little gem when we were i...  
4    Great quality ingredients, reasonable prices a...  
..                                                 ...  
261         